## 5.6 Affine/Softmaxレイヤの実装

前のセクションまでで乗算、加算、ReLU, Sigmoidといった計算レイヤの逆伝播を実装してきた。

参考： https://sge.qiita.com/yoshinari_yuto/items/e07fc89c36e123a58b49

このセクションからニューラルネットワークの逆伝播を実装していく。具体的には、行列の積（幾何学でアフィン変換とよばれる）における逆伝播を考える。

### 5.6.1 Affineレイヤ

ニューラルネットワークの順伝播は次のように実装した。

In [9]:
import numpy as np

X = np.random.rand(2) # 入力
W = np.random.rand(2, 3) # 重み
B = np.random.rand(3) # バイアス

Y = np.dot(X, W) + B

さらにこのYは活性化関数によって変換され、次の層へ伝播される。

これまでのように、ここで行った行列の積とバイアスの和を計算グラフで表してみる。

図5-24

この図は比較的単純なグラフだが、X, W, Bがスカラ値ではなく行列であることに注意して逆伝播を考えていく。

図5-25

最後のレイヤは単純に偏微分すれば良い。

$$
{\bf Y} = (y_1, y_2, y_3), \\
\frac{\partial L}{\partial {\bf Y}} =
\left ( \frac{\partial L}{\partial y_1}, \frac{\partial L}{\partial y_2}, \frac{\partial L}{\partial y_3} \right )
$$

次の加算レイヤの逆伝播はそのまま出力すればよかった。

$$
\frac{\partial L}{\partial ({\bf X} \cdot {\bf W})} = \frac{\partial L}{\partial {\bf Y}}, \\
\frac{\partial L}{\partial {\bf B}} = \frac{\partial L}{\partial {\bf Y}}
$$

最後のAffineレイヤ (アフィン変換を行う処理) は

$$
{\bf F} = {\bf X} \cdot {\bf W}
$$
と置いて、
$$
\frac{\partial L}{\partial {\bf X}} = \frac{\partial L}{\partial {\bf F}} \cdot \frac{\partial {\bf F}}{\partial {\bf X}} =
\frac{\partial L}{\partial ({\bf X} \cdot {\bf W})} \cdot \frac{\partial ({\bf X} \cdot {\bf W})}{\partial {\bf X}}
= \frac{\partial L}{\partial {\bf Y}} \cdot \frac{\partial ({\bf X} \cdot {\bf W})}{\partial {\bf X}}
$$
行列の積ができるように次元を調整して、
$$
\frac{\partial L}{\partial {\bf X}} = \frac{\partial L}{\partial {\bf Y}} \cdot {\bf W}^{\mathrm{T}}
$$

図5-26

Wについても同様に
$$
\frac{\partial L}{\partial {\bf W}} = {\bf X}^{\mathrm{T}} \cdot \frac{\partial L}{\partial {\bf Y}}
$$


### 5.6.2 バッチ版Affineレイヤ

前節で扱ったXはベクトルで、一つのデータを対象としていた。
次にN個のデータをまとめて順伝播する場合（バッチ）のAffineレイヤを考える。

図5-27

ここで、バイアス項Bの加算において行列の次元が異なるが、各データに対してWを加算することに注意する。
（N個のデータに対してAffine変換を実行すると考えれば自明）

計算例としては

In [10]:
X_dot_W = np.array([[0,0,0], [10,10,10]])
B = np.array([1,2,3])

X_dot_W

array([[ 0,  0,  0],
       [10, 10, 10]])

In [11]:
X_dot_W + B

array([[ 1,  2,  3],
       [11, 12, 13]])

このようにバイアスBの加算は各データに対して行われる。そのため、逆伝播の際には各データの逆伝播の値がバイアスに集約される。（そもそも次元も違うため、そうしなければ計算も合わない）

プログラムで表すと以下のようになる

In [12]:
dY = np.array([[1,2,3], [4,5,6]])
dY

array([[1, 2, 3],
       [4, 5, 6]])

In [13]:
dB = np.sum(dY, axis=0)
dB

array([5, 7, 9])

以上から、Affineレイヤの実装は以下のようになる

In [22]:
class Affine:
    def __init__(self, W, b):
        self.W = W
        self.b = b
        self.x = None
        self.dW = None
        self.db = None

    def forward(self, x):
        self.x = x
        out = np.dot(x, self.W) + self.b
        return out

    def backward(self, dout):
        dx = np.dot(dout, self.W.T) # .Tは転置
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis=0)
        return dx

### 5.6.3 Softmax-with-Lossレイヤ

最後に出力層のSoftmaxレイヤを実装していく。ソフトマックス関数については既に取り扱ったことがあるので、ここでの説明は省略する。

損失関数である交差エントロピー誤差 (cross entropy error) も含めて、Softmax-with-Lossレイヤという名前で実装する。
計算グラフは以下の図のようになる

図5-29

簡易的に表すと次の図のようになる

図5-30

ここでtはone-hotな教師ラベル（正解データ）である。まずCross entropy errorレイヤの逆伝播を考える。対数関数の微分は

$$
(\log (x) )' = \frac{1}{x}
$$

だったので、逆伝播を一つ一つ記入していくと下図のようになる

図A-4

よってCross entropy errorレイヤの逆伝播は

$$
\left ( -\frac{t_1}{y_1}, -\frac{t_2}{y_2}, -\frac{t_3}{y_3} \right )
$$

となる。続いてSoftmaxレイヤの逆伝播を考える。

ステップ1

まずCross entropy errorレイヤからの逆伝播の値が流れてくる。

ステップ2

最初の乗算ノードの片側（除算ノードに向かう方）は次のようになる

$$
-\frac{t_1}{y_1} \exp (a_1) = -\frac{t_1}{\frac{\exp (a_1)}{S}} \exp (a_1) = -t_1 S
$$

ステップ3

続けて除算の微分をする。この除算ノードは枝分かれして複数のノードに順伝播しており、バイアス項のときと同じように逆伝播では集約（各データを全て加算）される。
よって

$$
\left ( \frac{1}{f} \right )' = -\frac{f'}{f^2}
$$

なので、

$$
-S (t_1 + t_2 + t_3) \cdot \left ( \frac{1}{S} \right )' = S (t_1 + t_2 + t_3) \cdot \frac{1}{S^2} = \frac{1}{S} (t_1 + t_2 + t_3)
$$

ステップ4

またtはone-hotラベルであったので、その総和は1となり、このノードの逆伝播は

$$
\frac{1}{S}
$$

となる。加算ノードはそのまま流すだけ。

ステップ5

加算ノードのもう片方（expノードに向かう方）は

$$
-\frac{t_1}{y_1} \cdot \frac{1}{S} = -\frac{t_1}{\frac{\exp (a_1)}{S}} \cdot \frac{1}{S} = - \frac{t_1}{\exp(a_1)}
$$

となる。

ステップ6

最後にexpノードだが、expは微分しても変わらないので、

$$
\left ( \frac{1}{S} - \frac{t_1}{\exp(a_1)} \right ) \cdot (\exp (a_1))' = \left ( \frac{1}{S} - \frac{t_1}{\exp(a_1)} \right ) \cdot \exp (a_1)
= \frac{\exp (a_1)}{S} - t_1 = y_1 - t_1
$$

となる。以上から、順伝播の入力がa1のノードでは、逆伝播がy1-t1となることが導かれた。
逆伝播から最終的に得るベクトルは

$$
(y_1 - t_1, \, y_2 - t_2, \, y_3 - t_3)
$$

となる。これはたまたまきれいな結果が出たわけではなく、逆伝播が計算しやすくなるように交差エントロピー誤差が定義されている。

ここで具体例として、教師ラベルが (0,1,0) のようなデータに対して、Softmaxレイヤの出力が (0.3, 0.2, 0.5) 出会った場合を考える。
正解ラベルに対する確率は0.2なので、この時点では学習がまだまだ進んでいないことがわかる。
実際にこのレイヤの逆伝播は (0.3, -0.8, 0.5) となり、大きな誤差を伝播することになる。

重みを変えて学習を続け、教師ラベルが (0,1,0) のようなデータに対してSoftmaxレイヤの出力が (0.01, 0.99, 0) となった場合を考える。
この場合Softmaxレイヤからの逆伝播は (0.01, -0.01, 0) という小さな値になる。

Softmax-with-Lossレイヤの実装は以下のようになる。

In [20]:
from common.functions import softmax, cross_entropy_error

class SoftmaxWithLoss:
    def __init__(self):
        self.loss = None
        self.y = None # 順伝播の出力
        self.t = None # 教師データ (one-hot vector)

    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y, self.t)
        return self.loss

    def backward(self, dout=1):
        batch_size = self.t.shape[0]
        dx = (self.y - self.t) / batch_size
        return dx

ここで逆伝播の結果をバッチサイズで割っているが、データ1個あたりの誤差を伝播させるためにこうしている。

## 5.7 誤差逆伝播法の実装

これまで実装したレイヤを組み合わせることでニューラルネットワークの構築ができる。

### 5.7.1 ニューラルネットワークの学習の全体図

**前提**
```
ニューラルネットワークは、適応可能な重みとバイアスがあり、
この重みとバイアスを訓練データに適応するように調整することを「学習」とよぶ。
ニューラルネットワークの学習は次の4つの手順で行う。
```

**ステップ1（ミニバッチ）**
```
訓練データの中からランダムに一分のデータを取り出す。
```

**ステップ2（勾配の算出）**
```
各重みパラメータに関する損失関数の勾配を求める。
```

**ステップ3（パラメータの更新）**
```
重みパラメータを勾配方向に微小量だけ更新する。
```

**ステップ4（繰り返す）**
```
ステップ1~3を繰り返す。
```

第4章ではこの勾配を求めるために数値微分を利用したが、誤差逆伝播法を用いれば効率よく勾配を求めることができる。

### 5.7.2 誤差逆伝播法に対応したニューラルネットワークの実装

基本的には4章の実装と同じだが、ここではレイヤを保持する順序付きディクショナリ (OrderedDict) を定義している。
レイヤを使用することで、predictやgradientをレイヤの伝播だけで実装できるようになる。

2層のニューラルネットワーク TwoLayerNet を実装していくが、このクラスのインスタンス変数とメソッドを下表に整理しておく。

表5-1

表5-2

In [25]:
# coding: utf-8
import sys, os
sys.path.append(os.pardir)  # 親ディレクトリのファイルをインポートするための設定
import numpy as np
from common.layers import *
from common.gradient import numerical_gradient
from collections import OrderedDict


class TwoLayerNet:

    def __init__(self, input_size, hidden_size, output_size, weight_init_std = 0.01):
        # 重みの初期化
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size) 
        self.params['b2'] = np.zeros(output_size)

        # レイヤの生成
        self.layers = OrderedDict()
        self.layers['Affine1'] = Affine(self.params['W1'], self.params['b1'])
        self.layers['Relu1'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2'])

        self.lastLayer = SoftmaxWithLoss()
        
    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x) # レイヤの伝播
        
        return x
        
    # x:入力データ, t:教師データ
    def loss(self, x, t):
        y = self.predict(x)
        return self.lastLayer.forward(y, t)
    
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1) # one-hotラベルを通常のラベルに戻す
        if t.ndim != 1 : t = np.argmax(t, axis=1)
        
        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy
        
    # x:入力データ, t:教師データ
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)
        
        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        
        return grads
        
    def gradient(self, x, t):
        # forward
        self.loss(x, t)

        # backward
        dout = 1
        dout = self.lastLayer.backward(dout)
        
        layers = list(self.layers.values())
        layers.reverse() # 逆順でレイヤを伝播
        for layer in layers:
            dout = layer.backward(dout)

        # 設定
        grads = {}
        grads['W1'], grads['b1'] = self.layers['Affine1'].dW, self.layers['Affine1'].db
        grads['W2'], grads['b2'] = self.layers['Affine2'].dW, self.layers['Affine2'].db

        return grads

このようにニューラルネットワークの構成要素をレイヤとして実装したことで、ニューラルネットワークを簡単に実装することが出来た。

### 5.7.3 誤差逆伝播法の勾配確認

これまで勾配を求める方法を2つ取り扱ってきた。

- 数値微分によって求める方法
- 解析的に数式を解いて求める方法 (誤差逆伝播法)

数値微分による方法は計算に時間がかかるため学習で使われることはないが、実装が簡単なため、より複雑な誤差逆伝播法の実装をデバッグするときに利用できる。

数値微分の結果と誤差逆伝播法の結果を比較して、十分近い値が得られるか確認する作業を **勾配確認** (gradient check) という。

勾配確認の実装は次のようになる。

In [27]:
# coding: utf-8
import sys, os
sys.path.append(os.pardir)  # 親ディレクトリのファイルをインポートするための設定
import numpy as np
from dataset.mnist import load_mnist

# データの読み込み
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

x_batch = x_train[:3]
t_batch = t_train[:3]

grad_numerical = network.numerical_gradient(x_batch, t_batch)
grad_backprop = network.gradient(x_batch, t_batch)

for key in grad_numerical.keys():
    diff = np.average( np.abs(grad_backprop[key] - grad_numerical[key]) )
    print(key + ":" + str(diff))

W1:4.16578047279e-10
b1:2.45278031513e-09
W2:5.30048849398e-09
b2:1.39954044853e-07


上記のような実行結果が得られ、誤差が十分に小さく、よって誤差逆伝播法が正しく実装できていると判断できる。

### 5.7.4 誤差逆伝播法を使った学習

学習については4章でやった方法と同じなので省略する。

4章では数値微分によって勾配を求めたが、今回実装した誤差逆伝播法によって勾配を求めることに注意する。


## 5.8 まとめ

- 計算グラフを用いれば、計算過程を視覚的に把握することが出来る
- 計算グラフのノードは局所的な計算によって構成される。局所的な計算が全体の計算を構成する
- 計算グラフの順伝播は、通常の計算を行う。一方、計算グラフの逆伝播によって、各ノードの微分を求めることが出来る
- ニューラルネットワークの構成要素をレイヤとして実装することで、勾配の計算を効率的に求めることが出来る（誤差逆伝播法）
- 数値微分と誤差逆伝播法の結果を比較することで、誤差逆伝播法の実装に誤りがないことを確認できる（勾配確認）